In [131]:
from yahoo_oauth import OAuth2
import pandas as pd
import numpy as np
import yahoo_fantasy_api as yfa

#https://yahoo-fantasy-api.readthedocs.io/en/latest/yahoo_fantasy_api.html#the-league-class

In [113]:
conn = OAuth2(None, None, from_file='private.json')
if not conn.token_is_valid():
    conn.refresh_access_token()

[2020-08-16 17:26:02,184 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-08-16 17:26:02,186 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1051.5474870204926
[2020-08-16 17:26:02,187 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2020-08-16 17:26:02,189 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1051.5495719909668
[2020-08-16 17:26:02,189 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [151]:
league = yfa.League(conn,'nfl.l.254924')
free_agents = pd.DataFrame(league.free_agents('RB'))
##get the free agents
list_free_agents = list(free_agents['player_id'])

players = pd.DataFrame(league.player_stats(list_free_agents, 'season', season=2019))
#pd.DataFrame(league.player_details('Lamar Jackson'))
players.head(2)


,player_id,name,position_type,Pass Yds,Pass TD,Int,Rush Att,Rush Yds,Rush TD,Rec,Rec Yds,Rec TD,Ret TD,2-PT,Fum Lost,Fum Ret TD,Targets
0,7241,Frank Gore,O,0.0,0.0,0.0,166.0,599.0,2.0,13.0,100.0,0.0,0.0,0.0,0.0,0.0,16.0
1,8261,Adrian Peterson,O,0.0,0.0,0.0,211.0,898.0,5.0,17.0,142.0,0.0,0.0,0.0,2.0,0.0,23.0


In [174]:
# Grab season stats and calculate points
# Join points to prior season

season = 2019
prior_season = season - 1

points = {
    'Pass Yds': 0.04,
    'Pass TD': 4,
    'Int': -1,
    'Rush Att': 0.1,
    'Rush TD': 6,
    'Rec': 1,
    'Rec Yds': 0.1,
    'Rec TD': 6,
    'Ret TD': 6,
    '2-PT': 2,
    'Fum Lost': -2,
    'Fum Ret TD': 2
}

singleplayer = players # players.loc[players['player_id'] == 7241]
singleplayer['Points'] = singleplayer['Pass Yds'] * points['Pass Yds'] + singleplayer['Pass TD'] * points['Pass TD'] + singleplayer['Int'] * points['Int'] + singleplayer['Rush Att'] * points['Rush Att'] + singleplayer['Rush TD'] * points['Rush TD'] + singleplayer['Rec'] * points['Rec'] + singleplayer['Rec Yds'] * points['Rec Yds'] + singleplayer['Rec TD'] * points['Rec TD'] + singleplayer['Ret TD'] * points['Ret TD'] + singleplayer['2-PT'] * points['2-PT'] + singleplayer['Fum Lost'] * points['Fum Lost'] + singleplayer['Fum Ret TD'] * points['Fum Ret TD'] 

#singleplayer.sort_values('Points', ascending = False)

thisSeasonPoints = singleplayer[['player_id','Points']].copy()
thisSeasonPoints


,player_id,Points
58,30121,361.2
75,30423,266.5
71,30295,230.0
39,29238,206.1
59,30154,203.9
...,...,...
152,32630,0.0
78,30696,0.0
72,30301,0.0
203,33357,0.0
